In [1]:
import argparse
from memory_profiler import memory_usage


# parser is COMMENTED out because it doesn't work in jupyter notebook!!

########## START UNCOMMENT #################

# parser = argparse.ArgumentParser(description='Process some integers.')
# parser.add_argument('--condor',type=int, help="increase output verbosity",default = 0) 
# parser.add_argument('--ID',type=int,default = 404,
#                    help='the ID of the model and the logs of the dexpression run)')

# args = parser.parse_args()
# print("ID = " , args.ID)
# print("condor = " , args.condor)


# import sys
# if (args.condor > 0):
#     print ("Condor turned on")
#     sys.path.insert(0, '/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/python2.7/site-packages')
#     sys.path.insert(1,'/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/cuda_9.0/var/cuda-repo-9-0-local/usr/local/cuda-9.0/lib64')
#     sys.path.insert(2,'/esat/tiger/joramas/mscStudentsData/emotionModeling/libs/cudnn7/cudnn-9.0-linux-x64-v7/cuda/lib64')

#     print(sys.path)

# Give a run ID here. Change it to flags (arguments) in version 2.
ID = '4_1'

# if args.ID == None:
#     ID = '4_1'
# else:
#     ID = repr(args.ID)
#     print(args.ID)

########## STOP UNCOMMENT #################

RUNID = 'DeXpression_run_' + ID

import numpy as np
import tflearn
import tflearn.activations as activations
# Data loading and preprocessing
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d, global_avg_pool 
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.normalization import batch_normalization


#chris library imports
# from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split


from test_recursive_image_load_V2 import load_CKP_data
from test_recursive_image_load_V2 import load_formated_data
from test_recursive_image_load_V2 import split_dataset
from test_recursive_image_load_V2 import divide_subjects
from test_recursive_image_load_V2 import divide_data_to_subject
from test_recursive_image_load_V2 import load_npy_files

from showNumpyInfo import showInfo

from Dexpression_network import create_Dexpression_network
from Dexpression_network import create_Dexpression_GAP_network
from Dexpression_network import create_original_Dexpression_network

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:

# global Paths to define for each specific computer
#tf_checkpoints = where the checkpoints of tensorflow training algorithms are stored to be recovered if necessary
tf_checkpoints = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/tf_checkpoints"

# cascPath = the path to the cascade file for the facerecognition (relative paths didn't work on my windows edition)
cascPath = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/haarcascade.xml"

# Give a dropout if required (change to True and define the dropout percentage).
dropout_keep_prob=0.5

# Load data from: https://drive.google.com/drive/folders/1YWT8DJivNOZzQRPCiHDPY0LL_dymdQIS?usp=sharing
# X_data = np.load('../data/CKP_X.npy')
# Y_data = np.load('../data/CKP_Y.npy')
# X_subID = (np.load('../data/CKP_subjectIds.npy')).astype('uint8')

# X_data = np.load('../data/CKP_X_all.npy')
# Y_data = np.load('../data/CKP_Y_all.npy')
# X_subID = (np.load('../data/CKP_subjectIds_all.npy')).astype('uint8')

# load_dir = 'CKP_all_neutral'

# [X_data,Y_data,X_subID] = load_npy_files(5,load_dir)
# data = [X_data,Y_data,X_subID]

# print("CHECK data type and shape")

# print("Type of data var ", type(data))
# print("Type of data[0] ",  type(data[0]))

# showInfo(X_data,"X_data")
# showInfo(Y_data,"Y_data")
# showInfo(X_subID ,"X_subID")
# print("-----DONE DEBUG---------")



In [3]:
#load the subject distribution over the different datasets
# subID = (np.load('data_division/train_subject_ID.npy')).astype('uint8')
# subID_val = (np.load('data_division/validation_subject_ID.npy')).astype('uint8')
# subID_test = (np.load('data_division/test_subject_ID.npy')).astype('uint8')


# subID = (np.load('./data_division/'+load_dir+'/train_subject_ID.npy')).astype('uint8')
# subID_val = (np.load('./data_division/'+load_dir+'/validation_subject_ID.npy')).astype('uint8')
# subID_test = (np.load('./data_division/'+load_dir+'/test_subject_ID.npy')).astype('uint8')
# subIDs = [subID, subID_val, subID_test]

# print("CHECK subID type and shape")

# print("Type of data var ", type(data))
# print("Type of data[0] ",  type(data[0]))
# print("Type of subIDs var ", type(subIDs))
# print("Type of subIDs[0] ",  type(subIDs[0]))
# showInfo(subID,"subID")
# showInfo(subID_val,"subID_val")
# showInfo(subID_test ,"subID_test")
# print('the sum of instances in all the subID arrays', subID.shape[0]+subID_val.shape[0]+subID_test.shape[0] ,' needs to be the same as the first dimension of the X_data ', X_data.shape[0]  )
# print("-----DONE DEBUG---------")7

load_dir = 'CKP'

X_train = np.load('./generation/predivided_data/'+load_dir+'/X_train.npy')
Y_train = np.load('./generation/predivided_data/'+load_dir+'/Y_train.npy')

X_val = np.load('./generation/predivided_data/'+load_dir+'/X_val.npy')
Y_val = np.load('./generation/predivided_data/'+load_dir+'/Y_val.npy')

X_test = np.load('./generation/predivided_data/'+load_dir+'/X_test.npy')
Y_test = np.load('./generation/predivided_data/'+load_dir+'/Y_test.npy')

print( "tot length ", len(X_train) + len(X_val) + len(X_test))

data = [X_train , Y_train, X_val, Y_val, X_test, Y_test]



#monitor the memory usage of the current proces for 1 second, sample each 0.2 s return list of MB values
mem_usage = memory_usage(-1, interval=.2, timeout=1)
print('Maximum memory usage: %s' % mem_usage)


tot length  327
Maximum memory usage: [211.64453125, 211.64453125, 211.64453125, 211.64453125, 211.64453125]


In [4]:
import time


# def logprint(stre):
# 	print(stre)
# 	logging.info(stre)

# def getTimeSince(start,str):
#     end = time.time()
#     print(str)
#     print("time(sec) ",end - start)
#     return end
    
# def printInfo(i, text,start):   
#     if(i%50 == 0):
#         logprint(text)
#         stre =" "+ repr(i) + " images have been processed"
#         logprint(stre)
#         #monitor the memory usage of the current proces for 1 second, sample each 0.2 s return list of MB values
#         mem_usage = memory_usage(-1, interval=.2, timeout=1)
#         str = 'Maximum memory usage: ' + repr( mem_usage)
#         logprint(str)
#         getTimeSince(start,text)
#         return end


In [5]:

# start = time.time()
# print("dividing the data")
# divided_data = divide_data_to_subject(data,subIDs)

# X = (divided_data[0].reshape(-1,224,224,1)).astype('uint8')
# Y = (divided_data[1].reshape(-1,7)).astype('uint8')

# # create the validation set X_val and Y-val (SubID_val is not given to the network)
# X_val = divided_data[2].reshape(-1,224,224,1).astype('uint8')
# Y_val = divided_data[3].reshape(-1,7).astype('uint8')

# # create the test set X_test and Y_test (SubID_test is not given to the network)
# X_test = divided_data[4].reshape(-1,224,224,1).astype('uint8')
# Y_test = divided_data[5].reshape(-1,7).astype('uint8')

showInfo(X_train,"X_train")
showInfo(Y_train,"Y_train")
print('--------------------------')
showInfo(X_val,"X_val")
showInfo(Y_val,"Y_val")
print('--------------------------')
showInfo(X_test,"X_test")
showInfo(Y_test,"Y_test")


Name  X_train
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (205, 224, 224, 1)
Name  Y_train
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (205, 7)
--------------------------
Name  X_val
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (103, 224, 224, 1)
Name  Y_val
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (103, 7)
--------------------------
Name  X_test
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (19, 224, 224, 1)
Name  Y_test
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (19, 7)


In [6]:
# seperate the dataset in 11 parts that do not share users: 
#   9 parts train set
#   1 part validation set
#   1 part test set
# allows future possibility for 10-fold crossover

# parts = split_dataset([X_reduced,Y_reduced,X_subID],11)

# X_parts = parts[0]
# Y_parts = parts[1]
# X_subID_parts = parts[2]


In [7]:
#select 9 from the 11 parts (part 11 is usually the smallest so it is bad as test or validation data)
# select =  [0,1,2,3,6,7,8,9,10]
# select_val = [4]
# select_test= [5]

# selection = [select, select_val, select_test]

# if((len(select)+1 + 1)!= len(X_parts)):
#     print("Warning it is possible that not all parts of the dataset are used")


In [8]:
# divide the subjects from a database according to the division in selection
# IN:
# X_subID_parts = the subject labels of the data set split in to N parts
# selection = a list with 3 lists [select select_val select_test]
# OUT:
# A list of 3 lists with subject numbers for training, validation and test sets
# subIDs =  divide_subjects(X_subID_parts,selection)


In [9]:
def train_on_predivided_dexpression_model(RUNID,data,tf_checkpoints,cascPath,dropout_keep_prob=0.5):
    
    print("CHECK data types and shapes")
    print("length of data list: ",len(data))
    print("Type of data var ", type(data))
    print("Type of data[0] ",  type(data[0]))

    
    showInfo(data[0],"X_train")
    showInfo(data[1],"Y_train")
    showInfo(data[2],"X_val")
    showInfo(data[3],"Y_val")
    showInfo(data[4],"X_test")
    showInfo(data[5],"Y_test")

    print("-----DONE DEBUG---------")
    

    X = (data[0].reshape(-1,224,224,1)).astype('uint8')
    Y = (data[1].reshape(-1,7)).astype('uint8')

    # create the validation set X_val and Y-val (SubID_val is not given to the network)
    X_val = data[2].reshape(-1,224,224,1).astype('uint8')
    Y_val = data[3].reshape(-1,7).astype('uint8')

    # create the test set X_test and Y_test (SubID_test is not given to the network)
    X_test = data[4].reshape(-1,224,224,1).astype('uint8')
    Y_test = data[5].reshape(-1,7).astype('uint8')

#     network = create_Dexpression_network(dropout_keep_prob)

#     network with GAP layer
    network = create_original_Dexpression_network(dropout_keep_prob)
    


    #create a custom tensorflow session to manage the used resources
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config = config)


    # Final definition of model checkpoints and other configurations
    #model = tflearn.DNN(network, checkpoint_path='/home/cc/DeXpression/DeXpression_checkpoints',
    model = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")


    # Fit the model, train for 20 epochs. (Change all parameters to flags (arguments) on version 2.)
    #model.fit(X, Y, n_epoch=20, validation_set=0.1, shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)
    model.fit(X, Y, n_epoch=20, validation_set=(X_val,Y_val), shuffle=True, show_metric=True, batch_size=50, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)

    # Save the model
    model.save(tf_checkpoints + '/' + RUNID + '.model')
    print("finished training and saving")



In [10]:
train_on_predivided_dexpression_model(RUNID,data,tf_checkpoints,cascPath,dropout_keep_prob)

Training Step: 99  | total loss: 0.73722 | time: 7.756s
| Adam | epoch: 020 | loss: 0.73722 - acc: 0.8322 -- iter: 200/205
Training Step: 100  | total loss: 1.06384 | time: 11.031s
| Adam | epoch: 020 | loss: 1.06384 - acc: 0.7710 | val_loss: 3.46340 - val_acc: 0.4175 -- iter: 205/205
--
finished training and saving


In [11]:
# save the array with the Id's of each user per set, for future reference
# np.save('data_division/train_subject_ID.npy',subID)
# np.save('data_division/validation_subject_ID.npy',subID_val)
# np.save('data_division/test_subject_ID.npy',subID_test)


# just test outpt

# showInfo(X,"X")
# showInfo(Y,"Y")
# showInfo(X_val,"X_vall")
# showInfo(Y_val,"Y_vall")
# showInfo(X_test,"X_test")
# showInfo(Y_test,"Y_test")

# for i in range(0,len(X)):
#     str = "i " + repr(i) + " emo " +  repr(Y[i]) + " X  "+ repr(subID[i]) 
#     print(str)
#     cv2.imshow(str, X[i].reshape((224,224)))
#     cv2.namedWindow(str,cv2.WINDOW_NORMAL)
#     cv2.resizeWindow(str, 600,600)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
# showInfo(X , "X")

# cv2.imshow("example", X[1].reshape((224,224)))
# cv2.waitKey(0)

# print("size labels shape: " + repr(Y.shape))
# print("type labels: " + repr(type(Y)))


In [13]:
# def create_Dexpression_network():
#     # Define number of output classes.
#     num_classes = 7

#     # Define padding scheme.
#     padding = 'VALID'

#     # Model Architecture
#     network = input_data(shape=[None, 224, 224, 1])
#     conv_1 = relu(conv_2d(network, 64, 7, strides=2, bias=True, padding=padding, activation=None, name='Conv2d_1'))
#     maxpool_1 = batch_normalization(max_pool_2d(conv_1, 3, strides=2, padding=padding, name='MaxPool_1'))
#     LRN_1 = local_response_normalization(maxpool_1, name='LRN_1')
#     # FeatEX-1
#     conv_2a = relu(conv_2d(maxpool_1, 96, 1, strides=1, padding=padding, name='Conv_2a_FX1'))
#     maxpool_2a = max_pool_2d(maxpool_1, 3, strides=1, padding=padding, name='MaxPool_2a_FX1')
#     conv_2b = relu(conv_2d(conv_2a, 208, 3, strides=1, padding=padding, name='Conv_2b_FX1'))
#     conv_2c = relu(conv_2d(maxpool_2a, 64, 1, strides=1, padding=padding, name='Conv_2c_FX1'))
#     FX1_out = merge([conv_2b, conv_2c], mode='concat', axis=3, name='FX1_out')
#     # FeatEX-2
#     conv_3a = relu(conv_2d(FX1_out, 96, 1, strides=1, padding=padding, name='Conv_3a_FX2'))
#     maxpool_3a = max_pool_2d(FX1_out, 3, strides=1, padding=padding, name='MaxPool_3a_FX2')
#     conv_3b = relu(conv_2d(conv_3a, 208, 3, strides=1, padding=padding, name='Conv_3b_FX2'))
#     conv_3c = relu(conv_2d(maxpool_3a, 64, 1, strides=1, padding=padding, name='Conv_3c_FX2'))
#     FX2_out = merge([conv_3b, conv_3c], mode='concat', axis=3, name='FX2_out')
#     net = flatten(FX2_out)
#     if dropout:
#         net = dropout(net, dropout_keep_prob)
#     loss = fully_connected(net, num_classes,activation='softmax')

#     # Compile the model and define the hyperparameters
#     network = tflearn.regression(loss, optimizer='Adam',
#                          loss='categorical_crossentropy',
#                          learning_rate=0.0001)
#     return network

In [14]:
tf.RunOptions(report_tensor_allocations_upon_oom = True)

report_tensor_allocations_upon_oom: true

In [15]:
# cv2.imshow("example", X[1].reshape((224,224)))
# cv2.waitKey(0)


# Load the model if required, later.
#model.load('./DeXpression_checkpoints/' + RUNID + '.model')